# 1. Get Initialized Data

In [4]:
from pprint import pprint
from nlptext.base import BasicObject

########### BOSON ###########
CORPUSPath = 'corpus/boson/'
corpusFileIden = '.txt'
textType   = 'line'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

MaxTextIdx = False

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW, MaxTextIdx)

from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


'File'
corpus/boson/bosonNER.txt


Loading model cost 0.598 seconds.
Prefix dict has been built succesfully.


Total Num of All    Tokens 534211
The Total Number of Tokens: 534211
Counting the number unique Tokens...          	 2019-06-08 17:06:30.443100
		Done!
Generating Dictionary of Token Unique...	 2019-06-08 17:06:30.608856
		The length of DTU is: 3870 	 2019-06-08 17:06:30.609617
Generating the ORIGTokenIndex...       	 2019-06-08 17:06:30.609681
		The idx of token is: 0 	 2019-06-08 17:06:30.609976
		Done!
Only Keep First 3500000 Tokens.
The coverage rate is: 0.0
Total Num of Unique Tokens 3870
['</pad>', '</start>', '</end>', 'O', 'company_name-B', 'company_name-E', 'company_name-I', 'company_name-S', 'location-B', 'location-E', 'location-I', 'location-S', 'org_name-B', 'org_name-E', 'org_name-I', 'org_name-S', 'person_name-B', 'person_name-E', 'person_name-I', 'person_name-S', 'product_name-B', 'product_name-E', 'product_name-I', 'product_name-S', 'time-B', 'time-E', 'time-I', 'time-S']
CORPUS	it is Dumped into file: data/boson/char/Token3870/Pyramid/CORPUS.p
CORPUS	the length of it i

In [6]:
BasicObject.TOKEN['ANNOTokenIndex']

array([20, 22, 22, ...,  3,  3,  3], dtype=uint32)

# 2. Special Tokens

---


#### 1. Special Token

`PADDING` is 0;

`START` is the start of a sentence;

`END` is the end of a sentence;

So, what is `START`'s channel grain?

Itself? Or Unknown?

These three tokens are present in every possible channels;


----



These is also an anther special token:

`UNK`: the special tokens that is not present in the DictToken.

The unk is present in token level channel.

If the channel is context-dependent, then the corresponding grains for unk token are not unk grain.

If the channel is context-independent, then the corresponding grains for unk token are unk grains.


---

#### 2. Interpreting Dict: String to Index Dicts

DictToken is TokenString to Index



In [ ]:

PAD   = '</pad>'
UNK   = '</unk>'
START = '</start>'
END   = '</end>'

specialTokens     = [ PAD, UNK, START, END]
specialTokensDict = {PAD: 0, START: 1, END: 2, UNK : 3, }


# 3. Get Universe Grain List from a Token or a Sentence

**Helper Function**



Main module:

    `utils/channel.py`


In [ ]:


def grainChar(char):
    pass 



def getGrainNgrams(subword_infos, n):
    # Here N is the Num for n_gram
    #     subword_infos: [subcomp1, subcomp2, ...] or [stroke1, stroke2, ...]
    #                 n: the targeted n gram
    if n == 1:
        return [i for i in subword_infos]
    if n > len(subword_infos):
        # How to deal this when the length is not so long
        # Condition: where n is larger than the infos
        
        return [] # this or below?
        # return [UNK] 

    l = [subword_infos[i:n+i] for i in range(len(subword_infos) - n + 1)]
    l = ['-'.join(i) for i in l]
    return l

def grainToken(token, grainCharFunction, Ngram = 1,Max_Ngram = None, end_grain = True):
    '''
        token level only!
        The input token is not in Special Tokens
        The input token is a string!
        TODO: handle the `ngram` problems here.
        Content-Idenpendent Only
    '''
    if token not in specialTokens:
        infos = sum([grainCharFunction(char, end_grain) for char in token], [])
        if not Max_Ngram:
            return getGrainNgrams(infos, Ngram)
        else:
            return sum([getGrainNgrams(infos, idx+1) for idx in range(Max_Ngram)], [])
    else:
        return getGrainNgrams([token], Ngram) # deal with the special tokens
    
    
def grainSent_ctxInd(sent, grainCharFunction, Ngram = 1, Max_Ngram = None, end_grain = True):
    '''
        TODO: to update with context-dependent channel method
    '''
    return [grainToken(token, grainCharFunction, Ngram, Max_Ngram, end_grain) for token in sent]
    


## 3.1 CASE `basic`


### context-independent. A Token's Grain List (May More Than One Grain Element)



In [ ]:
import string
import re


def basicGrainChar(char, end_grain = False):
    '''
        Char level only, char is 字符 in the computational languages
        The input char is a string!
        This is only for context-independent channels
    '''
    punStr = string.punctuation + '＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､\u3000、〃〈〉《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏﹑﹔·！？｡。'
    engReg = r'[A-Za-z]{1}'
    if '%' in char or '%' in char:
        info =  'PERC'
    elif re.match(r'[0-9]{1}', char):
        info = "NUM"
    elif char in punStr:
        info = "PUNC"
    elif char >= '\u4e00' and char <= '\u9fff':
        info = "CHN"
    elif re.match(engReg, char):
        info = 'ENG'
    #elif word in string.whitespace:
        #return 'SPA'
    elif char == '@':
        info = 'SPA'
    else:
        info = 'OTHER'
    info = [info]
    if end_grain:
        info = info + ['b0']
    return info
    
char = '病'
token = '病理性'
sent = [START] + '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ') + [END]
print(''.join(sent))


end_grain = False
print(basicGrainChar(char))
print(grainToken(token, basicGrainChar,                   end_grain=end_grain))
print(grainSent_ctxInd(sent, basicGrainChar,              end_grain=end_grain))
print(grainSent_ctxInd(sent, basicGrainChar, Ngram=2,     end_grain=end_grain)) 
print(grainSent_ctxInd(sent, basicGrainChar, Max_Ngram=2, end_grain=end_grain)) 



## 3.2 CASE `radical`


### context-independent. A Token's Grain List (May More Than One Grain Element)


In [ ]:
import pickle

with open('nlptext/sources/CharRadical.p', 'rb') as handle:
    CharRadicalInfos = pickle.load(handle)

def radicalGrainChar(char, end_grain = False):
    '''
        char level only!
    '''
    if char in CharRadicalInfos:
        info = CharRadicalInfos[char]
        if info:
            info = [info] # here Radical Data is in str not list
        else:
            info = [char]
    else:
        info = [char]
        
    if end_grain:
        info = info + ['r0']
    return info

char = '病'
token = '病理性'
sent = [START] + '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ') + [END]
print(''.join(sent))


end_grain = False
print(radicalGrainChar(char), '\n')

print(grainToken(token,      radicalGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, radicalGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, radicalGrainChar, Ngram=2,     end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, radicalGrainChar, Max_Ngram=2, end_grain=end_grain), '\n')




## 3.3 CASE `pos`


### context-dependent. A Token's Grain List (May More Than One Grain Element)



In [ ]:
from jieba import posseg

def POSGrainSent(sent, tokenLevel = 'word', useStartEnd = True, tagScheme = 'BIO'):
    '''
        Here only for Atom is Char Based
        This method should be enriched

        sent: List of Token(String), with or without Start or End
        
    '''
    # d = []
    if useStartEnd:
        sent = sent[1:-1]
    segs = list(posseg.cut(''.join(sent)))
    
    GrainSent = []
    
    if tokenLevel == 'word':
        for i in range(len(segs)):
            pair  = segs[i]
            label = pair.flag
            GrainSent.append([label])
            # print(pair.word)
        if useStartEnd:
            return [[START]] + GrainSent + [[END]]
        return GrainSent
    elif tokenLevel == 'char':
        for i in range(len(segs)):
            '''
            pair  = segs[i]
            start = sum(len(p.word) for p in segs[:i])
            end   = sum(len(p.word) for p in segs[:i+1])
            label = pair.flag
            d.append([pair.word, start, end, label]) # still: SSET
            '''
            pair  = segs[i]
            leng  = len(pair.word)
            label = pair.flag
            labels= [label + '-I' ]*leng
            labels[0] = label + '-B'
            if 'E' in tagScheme and leng >= 2:
                labels[-1] = label + '-E'
            if 'S' in tagScheme and leng == 1:
                labels[0] = label + '-S'
            GrainSent.extend([[i] for i in labels])
        if useStartEnd:
            return [[START]] + GrainSent + [[END]]
        return GrainSent
    
print(POSGrainSent(sent, tokenLevel='word')                   , '\n')
print(POSGrainSent(sent, tokenLevel='char', tagScheme='BIEO') , '\n')
print(POSGrainSent(sent, tokenLevel='char', tagScheme='BIO')  , '\n')
print(POSGrainSent(sent, tokenLevel='char', tagScheme='BIEOS'), '\n')

In [ ]:

def grainSent_ctxDep(sent, channelGrainSent, tokenLevel = 'word', tagScheme = 'BIO', useStartEnd = True):
    return channelGrainSent(sent, tokenLevel=tokenLevel, tagScheme=tagScheme, useStartEnd = useStartEnd)


char = '病'
token = '病理性'
sent = [START] + '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ') + [END]
print(''.join(sent))


pprint(sent)
print(len(sent))
grainSentFunction = POSGrainSent
tokenLevel = 'word'
tagScheme = 'BIEO'


######## STARS #########
posResult = grainSent_ctxDep(sent, grainSentFunction, tokenLevel =tokenLevel , tagScheme = tagScheme)


pprint(posResult)
print(len(posResult))

# 4. Other Context-Ind Channels

## 4.1 CASE: `subcomp`

In [ ]:
import pickle

with open('nlptext/sources/CharSubComp.p', 'rb') as handle:
    CharSubCompInfos = pickle.load(handle)

def subcompGrainChar(char, end_grain = False):
    '''
        char level only!
    '''
    if char in CharSubCompInfos:
        info = CharSubCompInfos[char]
        if info:
            info = info 
        else:
            info = [char] 
    else:
        info = [char]
        
    if end_grain:
        info = info + ['c0']
    return info

char = '病'
token = '病理性'
sent = [START] + '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ') + [END]
print(''.join(sent))



end_grain = True
print(subcompGrainChar(char), '\n')

print(grainToken(token,      subcompGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, subcompGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, subcompGrainChar, Ngram=2,     end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, subcompGrainChar, Max_Ngram=2, end_grain=end_grain), '\n')


## 4.2 CASE: `stroke`

In [ ]:
import pickle


with open('nlptext/sources/CharStroke.p', 'rb') as handle:
    CharStrokeInfos = pickle.load(handle)
    
    
def strokeGrainChar(char, end_grain = False):
    '''
        char level only!
    '''
    if char in CharStrokeInfos:
        info = CharStrokeInfos[char]
        if info:
            info = info 
        else:
            info = [char] 
    else:
        info = [char]
        
    if end_grain:
        info = info + ['r0']
    return info

char = '病'
token = '病理性'
sent = [START] + '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ') + [END]
print(''.join(sent))


end_grain = True
print(strokeGrainChar(char), '\n')

print(grainToken(token,      strokeGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, strokeGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, strokeGrainChar, Ngram=2,     end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, strokeGrainChar, Max_Ngram=2, end_grain=end_grain), '\n')



## 4.3 CASE: `medical`

In [ ]:
# Medical Dictionary Tag
def medicalGrainChar(word, end_grain = False):
    # TODO
    units = 'kBq kbq mg Mg UG Ug ug MG ml ML Ml GM iu IU u U g G l L cm CM mm s S T % % mol mml mmol MMOL HP hp mmHg umol ng'.split(' ')
    chn_units = '毫升 毫克 单位 升 克 第 粒 颗粒 支 件 散 丸 瓶 袋 板 盒 合 包 贴 张 泡 国际单位 万 特充 个 分 次'.split(' ')
    med_units = 'qd bid tid qid qh q2h q4h q6h qn qod biw hs am pm St DC prn sos ac pc gtt IM IV po iH'.split(' ')
    all_units = units + chn_units + med_units

    site_units = '上 下 左 右 间 片 部 内 外 前 侧 后'.split(' ')
    sym_units = '大 小 增 减 多 少 升 降 高 低 宽 厚 粗 两 双 延 长 短 疼 痛 终 炎 咳'.split(' ')
    part_units = '脑 心 肝 脾 肺 肾 胸 脏 口 腹 胆 眼 耳 鼻 颈 手 足 脚 指 壁 膜 管 窦 室 管 髋 头 骨 膝 肘 肢 腰 背 脊 腿 茎 囊 精 唇 咽'.split(' ')
    break_units = '呈 示 见 伴 的 因'.split(' ')
    more_units = '较 稍 约 频 偶 偏'.split(' ')
    non_units = '无 不 非 未 否'.split(' ')
    tr_units = '服 予 行'.split(' ')

    if word in units:
        info =  'UNIT'
    elif word in chn_units:
        info =  'CHN_UNIT'
    elif word in med_units:
        info =  'MED_UNIT'
    elif word in site_units:
        info =  'SITE_UNIT'
    elif word in sym_units:
        info =  'SYM_UNIT'
    elif word in part_units:
        info =  'PART_UNIT'
    elif word in break_units:
        info =  'BREAK_UNIT'
    elif word in more_units:
        info =  'more_UNIT'
    elif word in non_units:
        info =  'NON_UNIT'
    elif word in tr_units:
        info =  'TR_UNIT'
    else:
        info =  'OTHER'
        
    # return [info]
    info = [info]
    if end_grain:
        info = info + ['m0']
    return info
    

char = '病'
token = '病理性'
sent = [START] + '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ') + [END]
print(''.join(sent))


end_grain = False
print(medicalGrainChar(char), '\n')

print(grainToken(token,      medicalGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, medicalGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, medicalGrainChar, Ngram=2,     end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, medicalGrainChar, Max_Ngram=2, end_grain=end_grain), '\n')



In [ ]:

def charGrainChar(char, end_grain = False):
    '''
        char level only!
    '''
    info = [char]
    if end_grain:
        info = info + ['ch0']
    return info


char = '病'
token = '病理性'
sent = [START] + '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ') + [END]
print(''.join(sent))


end_grain = False
print(medicalGrainChar(char), '\n')

print(grainToken(token,      medicalGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, medicalGrainChar,              end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, medicalGrainChar, Ngram=2,     end_grain=end_grain), '\n')
print(grainSent_ctxInd(sent, medicalGrainChar, Max_Ngram=2, end_grain=end_grain), '\n')




# 5. Other Context-Dep Channels

TODO

# 6. Wrapped Functions

In [ ]:

def getChannelGrain4Token(token, channel, Ngram = 1, Max_Ngram = None,  end_grain = False):
    '''
        token level only!
        The input token is not in Special Tokens
        The input token is a string!
        TODO: handle the `ngram` problems here.
        Content-Idenpendent Only
    '''

    if channel == 'token':
        return [token]

    if channel == 'basic':
        return grainToken(token, basicGrainChar,   Ngram = Ngram, Max_Ngram = Max_Ngram, end_grain = end_grain)
    
    if channel == 'medical':
        return grainToken(token, medicalGrainChar, Ngram = Ngram, Max_Ngram = Max_Ngram, end_grain = end_grain)
    
    if channel == 'radical':
        return grainToken(token, radicalGrainChar, Ngram = Ngram, Max_Ngram = Max_Ngram, end_grain = end_grain)
    
    if channel == 'subcomp':
        return grainToken(token, subcompGrainChar, Ngram = Ngram, Max_Ngram = Max_Ngram, end_grain = end_grain)
    
    if channel == 'stroke':
        return grainToken(token, strokeGrainChar,  Ngram = Ngram, Max_Ngram = Max_Ngram, end_grain = end_grain)
    
    if channel == 'char':
        return grainToken(token, charGrainChar,    Ngram = Ngram, Max_Ngram = Max_Ngram, end_grain = end_grain)
    
    else:
        print('The Channel "', channel, '" is not available currently!')

        
channel = 'stroke'
token = '四方'

getChannelGrain4Token(token, channel, Max_Ngram= 2, end_grain=False)


In [ ]:
CONTEXT_IND_CHANNELS = ['basic', 'medical', 'radical', 'token', 'char', 'subcomp', 'stroke']
CONTEXT_DEP_CHANNELS = ['pos']

def getChannelGrain4Sent(sent, channel, Ngram = 1, Max_Ngram = None, tokenLevel = 'char', tagScheme =  'BIO', useStartEnd = True, end_grain = False):
    '''
        token level only!
        The input token is not in Special Tokens
        The input token is a string!
        TODO: handle the `ngram` problems here.
        Content-Idenpendent Only
    '''
    if channel in CONTEXT_IND_CHANNELS:
        return [getChannelGrain4Token(token, channel, 
                                      Ngram = Ngram, 
                                      Max_Ngram = Max_Ngram, 
                                      end_grain = False) for token in sent]
        
    elif channel in CONTEXT_DEP_CHANNELS:
        if channel == 'pos':
            return grainSent_ctxDep(sent, POSGrainSent, tokenLevel =tokenLevel , tagScheme = tagScheme, useStartEnd = useStartEnd)
    

channel = 'pos'
sent = [START] + '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ') + [END]
print(''.join(sent))

getChannelGrain4Sent(sent, channel, tokenLevel = 'char')                   

# 7. Test

In [7]:
PAD   = '</pad>'
START = '</start>'
END   = '</end>'
UNK   = '</unk>'

specialTokens     = [ PAD, UNK, START, END]
specialTokensDict = {PAD: 0, START: 1, END: 2, UNK : 3, }

from nlptext.utils.channel import getChannelGrain4Token, getChannelGrain4Sent

ImportError: cannot import name 'getChannelGrain4Token' from 'nlptext.utils.channel' (/home/floydluo/Desktop/nlptext/nlptext/utils/channel.py)

In [ ]:
channel = 'radical'
token = '四方'

getChannelGrain4Token(token, channel)

In [ ]:
channel = 'radical'
sent = [START] + '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ') + [END]
getChannelGrain4Sent(sent, channel)                   

# 8. Test NLPText



In [ ]:
from pprint import pprint
from nlptext.base import BasicObject
from nlptext.utils import reCutText2Sent


########### ResumeNER ###########
CORPUSPath = 'dataset/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = reCutText2Sent
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW)

from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd

print([i for i in corpus.TOKEN])

## 8.1 Test Token

In [ ]:
tk = corpus.Tokens[35]
tk

In [ ]:
channel = 'radical'
tk.getChannelGrain(channel)

In [ ]:
channel = 'subcomp'
tk.getChannelGrain(channel, end_grain=True)

In [ ]:
channel = 'subcomp'

tk.getChannelGrain(channel, Ngram=2, end_grain=True)

In [ ]:
channel = 'stroke'
print(tk.getChannelGrain(channel, Max_Ngram=1, end_grain=False))

## 8.2 Test Sentence

In [ ]:
st = corpus.Sentences[2]
st

In [ ]:
channel = 'subcomp'

st.getChannelGrain(channel)

In [ ]:
channel = 'pos'
tagScheme = 'BIOES' # 'BIOE'
useStartEnd = True

st.getChannelGrain(channel, tagScheme = tagScheme, useStartEnd = useStartEnd)

In [ ]:
channel = 'annoE'
tagScheme = 'BIOE' # 'BIOE'
useStartEnd = False

annoE = st.getChannelGrain(channel, tagScheme = tagScheme, useStartEnd = useStartEnd)
list(zip(st.sentence, annoE))
# TODO, think more

 -- END --